This is apparently harder than I thought...so let's do it separately

In [51]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
import numpy as np
from scipy.stats import unitary_group
import matplotlib.pyplot as plt

from misc import mps_overlap
from state_approximation import mps2mpo, mpo2mps, diagonal_expansion, multiple_diagonal_expansions,\
    contract_diagonal_expansion, contract_series_diagonal_expansions
from disentanglers import disentangle_S2, renyi_entropy
from tebd import tebd
from state_approximation import entanglement_entropy

In [36]:
np.random.seed(0)

A0 = np.random.rand(2,2,2,2)
A1 = np.random.rand(2,2,2,2)
Lambda0 = np.random.rand(2,2,2,2)
Lambda1 = np.random.rand(2,2,2,2)

def contract(A0, A1, Lambda0, Lambda1):
    A = np.tensordot(A0, A1, [2,3])
    Lambda = np.tensordot(Lambda0, Lambda1, [2,3])
    combined = np.tensordot(A, Lambda, [[1,4],[0,3]])
    return combined

def contract_method_2(A0, A1, Lambda0, Lambda1):
    A_Lambda0 = np.tensordot(A0, Lambda0, [1,0])
    A_Lambda1 = np.tensordot(A1, Lambda1, [1,0])
    combined = np.tensordot(A_Lambda0, A_Lambda1, [[1,4],[2,5]])
    combined = np.transpose(combined, [0,1,4,5,2,3,6,7])
    return combined

out1 = contract(A0, A1, Lambda0, Lambda1)
out2 = contract_method_2(A0, A1, Lambda0, Lambda1)
np.allclose(out1, out2)

True

#### Applying to full A and Lambda 

In [38]:
tebd_state, _, _ = tebd(10, 1.5, 0.1)
Psi = mps2mpo(tebd_state.copy())
Lambda = Psi.copy()
As, Lambda, Ss, Lambdas = multiple_diagonal_expansions(Psi,1)
out = contract_series_diagonal_expansions(As, Lambda)
assert np.isclose(mps_overlap(Psi, out), 1.0)

A0 = As[0]
L = len(A0)

def split_tensor_svd(node, legs_left, legs_right):
    """ 
    Splits a tensor A using a QR decomposition. 
    Parameters
    ----------
    A : np.Array
        Tensor to be split
    legs_left : list of int
        The legs on the left.
    legts_right : legs on the right of the qr decomposition
    Returns
    -------
    q, r : np.Array
        Tensors with shapes (*legs_left, -1) and (-1, *legs_right).
        The leg order is given by legs_left and legs_right
    """
    perm = np.concatenate([legs_left, legs_right])
    shape_left = [node.shape[i] for i in legs_left]
    shape_right = [node.shape[i] for i in legs_right]
    node_shifted = node.copy().transpose(perm)
    node_shifted = node_shifted.reshape(np.prod(shape_left), np.prod(shape_right))
    
    
    u, s, v = np.linalg.svd(node_shifted, full_matrices=False)
    s = s[s > 1.e-10]
    chi_max = len(s)
    u = u[:,:chi_max]
    v = v[:chi_max,:]
    
    u = u.reshape(*shape_left, -1)
    v = (np.diag(s) @ v).reshape(-1, *shape_right)
    return u, v
    
def get_theta(Lambda, i):
    """ 
    Gets a TEBD style wavefunction on Lambda at index i, using
    the site at index i and the site immediately below it.
    Assumes a canonical form. 
    Parameters
    ----------
    Lambda : np.Array
        List of tensors, starting from bottom to top.
    i : int
        The index of the tensor to construct the tebd style
        wavefunction from
    """
    B0, B1 = Lambda[i], Lambda[i-1]
    contracted = np.tensordot(B0, B1, [2,3])
    p1, p2, chiN, p3, p4, chiS = contracted.shape
    theta = contracted.reshape(p1, p2*chiN, p3, p4*chiS).transpose([1,0,2,3])
    return theta

def disentangle_last_tensor(A0, Lambda, direction):
    """ 
    The last tensor isn't translation invariant, so this function
    takes care of that. Lambda0 and Lambda1 are the two site tensors
    from top to bottom.
    Parameters
    ----------
    A0 : list of np.Array
        Left column wavefunction
    Lambda : list of np.Array
        Right column wavefunction
    direction : str
        Either 'up' or 'down' depending on direction.
    """
    assert direction in ['up', 'down']
    L = len(Lambda)
    
    theta = get_theta(Lambda, L-1)
    theta, U = disentangle_S2(theta)
    Lambda_two_site = np.tensordot(Lambda[L-1], Lambda[L-2], [2,3])
    Lambda_two_site = np.tensordot(Lambda_two_site, U.conj(), [[0,3],[2,3]])
    
    if direction == 'down':
        legs_left, legs_right = [4,0,1], [5,2,3]
    else:
        legs_right, legs_left = [4,0,1], [5,2,3]        
    Lambda0_new, Lambda1_new = split_tensor_svd(Lambda_two_site, legs_left, legs_right)
    # TODO maybe make an RQ method so this isn't necessary
    if direction == 'down':
        Lambda0_new = Lambda0_new.transpose([0,1,3,2])
        Lambda1_new = Lambda1_new.transpose([1,2,3,0])
        Lambda[L-1] = Lambda0_new
        Lambda[L-2] = Lambda1_new
    else:
        Lambda1_new = Lambda1_new.transpose([1,2,0,3])
        Lambda[L-2] = Lambda0_new
        Lambda[L-1] = Lambda1_new
        
    A0[L-1] = np.tensordot(A0[L-1], U, [[1,3],[2,3]]).transpose([0,2,1,3])
    return A0, Lambda


def sweep_down_and_up(A0, Lambda):
    """ Sweeps down and up A0 and Lambda, disentangling at each step. """
    # Down sweep 
    A0, Lambda = disentangle_last_tensor(A0, Lambda, 'down')
    for i in range(L-2, 1, -1):
        theta = get_theta(Lambda, i)
        theta, U = disentangle_S2(theta)

        A = np.tensordot(A0[i+1], A0[i], [2,3])
        A = np.tensordot(A, U, [[1,4],[2,3]])
        
        # repeated but let's be careful 
        Lambda_contracted = np.tensordot(Lambda[i], Lambda[i-1], [2,3])
        Lambda_contracted = np.tensordot(Lambda_contracted, U.conj(), [[0,3],[2,3]])

        A0_new, A1_new = split_tensor_svd(A, [0,4,1], [2,5,3])
        A0_new = A0_new.transpose([0,1,3,2])
        A1_new = A1_new.transpose([1,2,3,0])

        Lambda0_new, Lambda1_new = split_tensor_svd(Lambda_contracted, [4,0,1], [5,2,3])
        Lambda0_new = Lambda0_new.transpose([0,1,3,2])
        Lambda1_new = Lambda1_new.transpose([1,2,3,0])

        A0[i+1] = A0_new
        A0[i] = A1_new

        Lambda[i] = Lambda0_new
        Lambda[i-1] = Lambda1_new

    #A0, Lambda = disentangle_last_tensor(A0, Lambda, 'down')
    print(entanglement_entropy(Lambda))
    for i in range(0, L-2):
        theta = get_theta(Lambda, i+1)
        theta, U = disentangle_S2(theta)


        A = np.tensordot(A0[i+2], A0[i+1], [2,3])
        A = np.tensordot(A, U, [[1,4],[2,3]])

        Lambda_contracted = np.tensordot(Lambda[i+1], Lambda[i], [2,3])
        Lambda_contracted = np.tensordot(Lambda_contracted, U.conj(), [[0,3],[2,3]])

        A0_new, A1_new = split_tensor_svd(A, [0,4,1], [2,5,3])
        A0_new = A0_new.transpose([0,1,3,2])
        A1_new = A1_new.transpose([1,2,3,0])

        Lambda0_new, Lambda1_new = split_tensor_svd(Lambda_contracted, [4,0,1], [5,2,3])
        Lambda0_new = Lambda0_new.transpose([0,1,3,2])
        Lambda1_new = Lambda1_new.transpose([1,2,3,0])

        A0[i+2] = A0_new
        A0[i+1] = A1_new

        Lambda[i+1] = Lambda0_new
        Lambda[i] = Lambda1_new

    A0, Lambda = disentangle_last_tensor(A0, Lambda, 'up')
    return(A0, Lambda)
import pickle
print(entanglement_entropy(Lambda))
with open("single_split_state.pkl", "wb+") as f:
    pickle.dump(Lambda, f)
for i in range(10):
    A0, Lambda = sweep_down_and_up(A0, Lambda)
    print(entanglement_entropy(Lambda))
    contracted = contract_diagonal_expansion(A0, Lambda)
    assert np.isclose(mps_overlap(Psi, contracted), 1.0)

0.24232390144270413
0.32296691099273034
0.3411149803293299
0.35815204216609353
0.385863235024431
0.3858961093830575
0.30936307894757986
0.3093599465913438
0.3950358117847723
0.3950382875835426
0.42738102209288
0.42806514123818634
0.42573226062165903
0.42568628090946403
0.3915090332371902
0.39168124492571377
0.44124709402103635
0.44124835310260835
0.41643726419812543
0.4164370478511884
0.44496693257795217


In [127]:
### That's not working let's try again

In [33]:
[i.shape for i in A0]

[(2, 1, 1, 2),
 (2, 2, 2, 8),
 (2, 2, 8, 32),
 (2, 2, 32, 128),
 (2, 2, 128, 32),
 (2, 2, 32, 8),
 (2, 2, 8, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2),
 (2, 2, 2, 2)]

In [48]:
def sweep_lambda(Lambda):
    
    for i in range(L-1, 0, -1):
        theta = get_theta(Lambda, i)
        p1, p2, _, chiN = Lambda[i].shape
        p3, p4, chiS, _ = Lambda[i-1].shape
        theta, U = disentangle_S2(theta, max_iter=2000)
        
        Lambda_contracted = theta.reshape(p2,chiN,p1,p3,p4,chiS).transpose([2,0,1,3,4,5])
        
        Lambda0_new, Lambda1_new = split_tensor_svd(Lambda_contracted, [0,1,2], [3,4,5])
        Lambda0_new = Lambda0_new.transpose([0,1,3,2])
        Lambda1_new = Lambda1_new.transpose([1,2,3,0])

        Lambda[i] = Lambda0_new
        Lambda[i-1] = Lambda1_new
    
    for i in range(1, L):
        theta = get_theta(Lambda, i)
        p1, p2, _, chiN = Lambda[i].shape
        p3, p4, chiS, _ = Lambda[i-1].shape
        theta, U = disentangle_S2(theta, max_iter=2000)
        
        Lambda_contracted = theta.reshape(p2,chiN,p1,p3,p4,chiS).transpose([2,0,1,3,4,5])
        
        Lambda0_new, Lambda1_new = split_tensor_svd(Lambda_contracted, [0,1,2], [3,4,5])
        Lambda0_new = Lambda0_new.transpose([0,1,3,2])
        Lambda1_new = Lambda1_new.transpose([1,2,3,0])

        Lambda[i] = Lambda0_new
        Lambda[i-1] = Lambda1_new
    return Lambda

def _sweep_disentangle(Psi):
    L = len(Psi)
    for i in range(L-1):
        theta = np.tensordot(Psi[i], Psi[i+1], [2,1]).transpose([1,0,2,3])
        theta, U = disentangle_S2(theta, max_iter=200, eps=1.e-16)
        chiL, d1, d2, chiR = theta.shape
        q, r = np.linalg.qr(theta.reshape(chiL*d1, chiR*d2))
        Psi[i] = q.reshape((chiL, d1, -1)).transpose([1,0,2])
        Psi[i+1] = r.reshape((-1, d2, chiR)).transpose([1,0,2])
    return Psi

def sweep_disentangle(Psi):
    Psi = [psi.transpose([0,2,1]) for psi in Psi[::-1]]
    Psi = _sweep_disentangle(Psi)
    #print(entanglement_entropy(Psi))
    Psi = [psi.transpose([0,2,1]) for psi in Psi[::-1]]
    Psi = _sweep_disentangle(Psi)
    #print(entanglement_entropy(Psi))
    return Psi

In [49]:
with open("single_split_state.pkl", "rb") as f:
    Lambda = pickle.load(f)
print(entanglement_entropy(Lambda))
Lambda = mpo2mps(Lambda)

0.24232390144270416


In [ ]:
print(entanglement_entropy(Lambda))
for i in range(10):
    Lambda = sweep_lambda(Lambda)
    print(entanglement_entropy(Lambda))
print(entanglement_entropy(Lambda))